# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Pytorch
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, models, transforms, utils

# Miscellaneous
import os
import time
import json
from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers/'

In [ ]:
batch_size = 16

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.RandomRotation(30),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(225),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


data_transforms = {'train': train_transforms,
                  'valid' : test_transforms,
                  'test' : test_transforms}

images_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform=data_transforms[x])
                   for x in ['train', 'valid', 'test']}

dataloaders = {x: torch.utils.data.DataLoader(images_datasets[x], batch_size=batch_size, shuffle=True)
               for x in ['train', 'valid', 'test']}

dataset_sizes = {x: len(images_datasets[x]) for x in ['train', 'valid', 'test']}

In [ ]:
for x in ['train', 'valid', 'test']:
    print(f'{x} dataset size: {dataset_sizes[x]}')

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

## Visualizing images

In [ ]:
# Visualize image data: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

def image_show(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

image_show(out, title=[cat_to_name[x] for x in cat_to_name])

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

### Densenet

In [ ]:
def load_model(hidden_units, arch, num_classes):
    model = getattr(models, arch)(pretrained=True)
    
    # Get the size of last hidden layer to rebuild new classifier
    if 'densenet' in arch:
        num_features = model.classifier.in_features
    elif 'resnet' in arch:
        num_features = model.fc.in_features
    elif 'vgg' in arch:
        # We want to add new layers to classifier, so get the last output size
        num_features = model.classifier[-1].out_features
    
    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
        
    # Define new classifier to classify 102 classes of flowers
    classifier = nn.Sequential(
        nn.Linear(in_features=num_features, out_features=hidden_units),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(in_features=hidden_units, out_features= num_classes),
        nn.LogSoftmax(dim=1)
    )
    
    if 'densenet' in arch:
        model.classifer = classifier
    elif 'resnet' in arch:
        model.fc = classifier
    elif 'vgg' in arch:
        # Unfreeze classifier because we just add new layers to it
        for param in model.classifier.parameters():
            param.requires_grad = True
        model.classifier = nn.Sequential(model.classifier, classifier)
    
    
    return model

### Train model

In [ ]:
if torch.cuda.is_available():
    use_gpu = True
    device = torch.device("cuda:0")
else:
    use_gpu = False
    device = torch.device("cpu")

In [ ]:
model = load_model(hidden_units=512, 'densenet121', num_classes=102)
model.to(device)

In [ ]:
def accuracy_score(model, data, criterion):
    loss = 0
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for X, y in data:
            if use_gpu:
                X, y = X.to(device), y.to(device)

            output = model.forward(X)
            loss += criterion(output, y).item()

            total += y.shape[0]
            correct += (y.data == output.max(dim=1)[1]).sum().item()


    return loss / len(data), correct / total 

In [ ]:
def train(model, train_data, criterion, optimizer, epochs, verbose=True):
    for e in range(epochs):
        model.train()
        epoch_loss = 0
        
        for X_train, y_train in dataloaders['train']:
            X_train, y_train = X_train.to(device), y_train.to(device)

            model.zero_grad()
            output = model.forward(X_train)
            loss = criterion(output, y_train)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        val_loss, val_acc = accuracy_score(model, dataloaders['valid'], criterion)
        
        if verbose:
            print("Epoch: {}/{}..  ".format(e + 1, epochs),
                  "Epoch loss: {:.4f}..  ".format(epoch_loss / len(dataloaders['train'])),
                  "Validation loss: {:.4f}..  ".format(val_loss),
                  "Validation Accuracy: {:.4f}..  ".format(val_acc))             

In [ ]:
# Define loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(params = model.classifier.parameters(),)

epochs = 9
train(model, dataloaders['train'], criterion, optimizer, epochs)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
test_loss, test_acc = accuracy_score(model, dataloaders['test'], criterion)
print(f'Test Accuracy: {test_acc * 100}%')

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
model.class_to_idx = images_datasets['train'].class_to_idx

In [ ]:
# TODO: Save the checkpoint 
checkpoint = "densenet121.pth"
state = {
    "epochs" : epochs,
    "batch_size" : batch_size,
    "clf_output_size" : 512,
    "opt_state_dict" : optimizer.state_dict(),
    "criterion" : criterion,
    "class_to_idx" : model.class_to_idx,
    "state_dict": model.state_dict()
}

torch.save(state, checkpoint)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = models.densenet121(pretrained=True)
    
    for params in model.parameters():
        params.require_grad = False
        
    num_features = model.classifier.in_features    
    classfier = nn.Sequential(
        nn.Linear(num_features, checkpoint['clf_output_size']),
        nn.ReLU(),
        nn.Dropout(0,2),
        nn.Linear(checkpoint['clf_output_size'], len(checkpoint['class_to_idx'])),
        nn.LogSoftmax(dim=1)
    )
    
    model.classifier = classfier
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model, checkpoint

In [ ]:
model, checkpoint = load_checkpoint('densenet121.pth')

In [ ]:
criterion = checkpoint['criterion']

In [ ]:
# Load optimizer state dict
# checkpoint['opt_state_dict']

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
from PIL import Image

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # Pre-process image
    image.thumbnail((256, 256))
    # CenterCrop: (left, upper, right, lower)-tuple
    w, h = image.size
    image = image.crop((w//2 - 224//2, h//2 - 224//2, w//2 + 224//2, h//2 + 224//2))
    image = np.array(image)
    
    # Normalize image
    means = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    
    image = (image / 255 - means) / std
    
    return image.T

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    plt.title(title)
    return ax

In [ ]:
image = Image.open('./flowers/test/1/image_06743.jpg')
image = process_image(image)
imshow(image, None, cat_to_name['1'])

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
class_to_idx = checkpoint['class_to_idx']

In [ ]:
idx_to_class = {value:key for key, value in class_to_idx.items()}

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    image = Image.open(image_path)
    image = torch.from_numpy(process_image(image))
    image = image.unsqueeze(0).type(torch.cuda.FloatTensor)
    image.to(device)
    
    model.eval()
    with torch.no_grad():
        output = model(image)
        exp_output = torch.exp(output)
        probs, max_classes = exp_output.data.topk(5)
        if use_gpu:
            max_classes = max_classes.cpu()
        max_idx = [idx_to_class[i] for i in max_classes.numpy()[0]]
        
    return probs, max_idx

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
def display(class_idx, model, num_sample=5, topk = 5):
    image_dir = './flowers/test/' + class_idx
    print(f'Class: {cat_to_name[class_idx]}')
    file_names = os.listdir(image_dir)
    for image_names in file_names:
        image_path = image_dir + '/' + image_names
        image = Image.open(image_path)
        probs, max_idx = predict(image_path, model, topk)
        
        # VISUALIZE RESULTS
        plt.figure(figsize=(16,7))
        fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols=2)
       
        # Display image
        imshow(process_image(image), ax1, cat_to_name[str(max_idx[0])])
        
        # Convert probs to numpy array
        if use_gpu:
            probs = probs.cpu()
        probs = probs.numpy()[0]      # .numpy() returns np-array of 2 elements, second of which is the data type
        y_pos = np.arange(len(probs)) # y axis
        y_values = [cat_to_name[idx] for idx in max_idx]
        
        # Bar Plot
        ax2.barh(y_pos, probs, align='center', color='b')
        ax2.set_yticks(y_pos)
        ax2.set_yticklabels(y_values)
        ax2.invert_yaxis()  # labels read top-to-bottom
        ax2.set_xlabel('Probabilities')
        ax2.set_title('Predictions')
        plt.tight_layout()
        
#         for i in max_idx:
#             print(f'{i}: {cat_to_name[i]}')
#         print('-'*15)

In [ ]:
display('1', model)

## Question

**Why does vgg models perform very poorly compared to others? Is that because of overfitting?**

**What is torch.tensor.data? How is that different from tensor?**